In [1]:
import pandas as pd
import numpy as np
import pickle
from Helper_functions import *

Using TensorFlow backend.


# Helper functions

In [2]:
def get_diggings(raw : pd.core.frame.DataFrame): # -> pd.core.frame.DataFrame
    practice = raw[["maleID","femaleID",'Liked', 'SexAtt', "LikYes", 'FLiked', 'FSexAtt', "FLikYes"]].copy()
    male_measure = ['Liked', 'SexAtt', "LikYes"]
    female_measure = ['FLiked', 'FSexAtt', "FLikYes"]
    practice.loc[:,"Digging"] = practice[male_measure].values.astype("float").sum(axis = 1)/len(male_measure)
    practice.loc[:,"FDigging"] = practice[female_measure].values.astype("float").sum(axis = 1)/len(female_measure)
    
    return practice

def get_transformations(practice : pd.core.frame.DataFrame, G : str): # -> pd.core.frame.DataFrame
    
    Gender = dict(
            M = "Digging ",
            F = "FDigging"
    )
    sort_by = dict(
         M = ["maleID","femaleID"],
         F = ["femaleID", "maleID"]
        )
    
    firstID = sort_by[G][0]
    secondID = sort_by[G][1]
    scores = Gender[G]
    # Digging grand mean  
    DGM = practice.loc[:,scores].mean() 

    
    #actor
    actor_data = a_p_score(practice, DGM, firstID,scores) # change practice could be better
    actor_df = pd.DataFrame(actor_data, columns = ["actor"])

    ids = practice[["maleID","femaleID"]].copy()
    ids.sort_values(by = firstID, inplace = True)
    ids.reset_index(inplace = True)

    actor_df[["maleID","femaleID"]] = ids.loc[:,["maleID","femaleID"]]

    # partner
    partner_data = a_p_score(practice, DGM, secondID, scores)
    partner_df = pd.DataFrame(partner_data, columns = ["partner"])

    # coordinate parner values with appropriate IDs
    ids = practice[["maleID","femaleID"]].copy()
    ids.sort_values(by = secondID, inplace = True)
    ids.reset_index(inplace = True)
    partner_df[["maleID","femaleID"]] = ids.loc[:,["maleID","femaleID"]]

    # join the male actor and parner df
    df = pd.merge(actor_df, partner_df, on = ["maleID", "femaleID"])

    # merge with practice
    practice = pd.merge(practice,df, on = ["maleID","femaleID"])

    digging = practice[scores].values.astype("float")
    actor = practice["actor"].values.astype("float")
    partner = practice["partner"].values.astype("float")

    relationship = digging - (DGM + actor + partner)
    
    # relationship
    practice["relationship"] = relationship
    
    return practice[["maleID","femaleID","actor","partner","relationship"]]

In [3]:
PATH = "/Users/oluwaseuncardoso/492_19summer/MachineLearning/RAW DATA/raw.csv"

In [4]:
missing_values = [" ", ""]
raw = pd.read_csv(PATH, na_values = missing_values)

/Users/oluwaseuncardoso/anaconda2/envs/AI/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (113,122,127,154,211,213,479,480,494,495,497,498,499,529,530,561,562,619,620,665,707,716,718,748,1073,1075,1213,1214,1216,1512) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
raw.columns

Index(['maleID', 'femaleID', 'code', 'revcode', 'rotation', 'class',
       'dateorder', 'Knew', 'Inter1', 'Liked',
       ...
       'DiggingActorGM', 'DiggingPartner', 'DiggingPartnerGM', 'DiggingRel',
       'FDiggingGrandMean', 'FDiggingActor', 'FDiggingActorGM',
       'FDiggingPartner', 'FDiggingPartnerGM', 'FDiggingRel'],
      dtype='object', length=1523)

In [6]:
practice = raw[["maleID","femaleID",'Liked', 'SexAtt', "LikYes", 'FLiked', 'FSexAtt', "FLikYes"]]
male_measure = ['Liked', 'SexAtt', "LikYes"]
female_measure = ['FLiked', 'FSexAtt', "FLikYes"]
practice.loc[:,"Digging"] = practice[male_measure].values.astype("float").sum(axis = 1)/len(male_measure)
practice.loc[:,"FDigging"] = practice[female_measure].values.astype("float").sum(axis = 1)/len(female_measure)

/Users/oluwaseuncardoso/anaconda2/envs/AI/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/Users/oluwaseuncardoso/anaconda2/envs/AI/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [9]:
# Digging grand mean
DGM = practice.loc[:,"Digging"].mean() 

#actor
actor_data = a_p_score(practice, DGM, "maleID","Digging") # change practice could be better
male_actor_df = pd.DataFrame(actor_data, columns = ["M_actor"])

ids = practice[["maleID","femaleID"]].copy()
ids.sort_values(by = "maleID", inplace = True)
ids.reset_index(inplace = True)

male_actor_df[["maleID","femaleID"]] = ids.loc[:,["maleID","femaleID"]]

# partner
partner_data = a_p_score(practice, DGM, "femaleID", "Digging")
female_partner_df = pd.DataFrame(partner_data, columns = ["F_partner"])

# coordinate parner values with appropriate IDs
ids = practice[["maleID","femaleID"]].copy()
ids.sort_values(by = "femaleID", inplace = True)
ids.reset_index(inplace = True)
female_partner_df[["maleID","femaleID"]] = ids.loc[:,["maleID","femaleID"]]

# join the male actor and parner df
male_df = pd.merge(male_actor_df, female_partner_df, on = ["maleID", "femaleID"])

# merge with practice
practice = pd.merge(practice,male_df, on = ["maleID","femaleID"])

digging = practice["Digging"].values.astype("float")
actor = practice["M_actor"].values.astype("float")
partner = practice["F_partner"].values.astype("float")

maler_relationship = digging - (DGM + actor + partner)
# relationship
practice["M_relationship"] = maler_relationship

In [10]:
raw[["maleID","femaleID","FDiggingActorGM","FDiggingPartnerGM","FDiggingRel"]].sort_values(by="femaleID").head()

,maleID,femaleID,FDiggingActorGM,FDiggingPartnerGM,FDiggingRel
999,381,7,-0.59,-1.79,1.15
600,277,7,-0.59,-0.73,0.76
387,210,7,-0.59,-1.82,0.18
423,215,7,-0.59,0.49,2.54
399,212,7,-0.59,-0.04,0.73


In [11]:
prac1 = get_diggings(raw)
prac2 = get_transformations(prac1, "F")
prac2.sort_values(by="femaleID").head()

,maleID,femaleID,actor,partner,relationship
999,381,7,-0.596217,-1.790661,1.151772
600,277,7,-0.596217,-0.735105,0.762883
387,210,7,-0.596217,-1.818439,0.179550
423,215,7,-0.596217,0.487117,2.540661
399,212,7,-0.596217,-0.040661,0.735105


In [13]:
prac1 = get_diggings(raw)
prac2 = get_transformations(prac1, "M")
prac2.sort_values(by="maleID").head()

,maleID,femaleID,actor,partner,relationship
0,15,13,-0.941049,-1.635494,0.607716
11,15,388,-0.941049,0.808951,-1.503395
10,15,127,-0.941049,0.975617,-0.670062
9,15,125,-0.941049,-1.829938,0.802160
7,15,90,-0.941049,1.614506,-1.308951


In [14]:
raw[["maleID","femaleID","DiggingActorGM","DiggingPartnerGM","DiggingRel"]].sort_values(by="maleID").head()

,maleID,femaleID,DiggingActorGM,DiggingPartnerGM,DiggingRel
0,15,13,-0.94,-1.64,0.61
11,15,388,-0.94,0.81,-1.50
10,15,127,-0.94,0.98,-0.67
9,15,125,-0.94,-1.83,0.80
7,15,90,-0.94,1.61,-1.31
